### Nonlinear Option Pricing, Master 2 Probabilités et Finance, Sorbonne Université and École polytechnique
# Homework II (optional)

### Due Date: 1:55 PM Tuesday, January 28, 2025
You should turn in the notebook on the Moodle course website.

Please comment your code properly.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly in a reasonable time frame, and then save the notebook.

In [23]:
pip install numpy==1.19.5

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [233 lines of output]
      setup.py:67: RuntimeWarning: NumPy 1.19.5 may not yet support Python 3.10.
        warnings.warn(
      Running from numpy source directory.
      setup.py:480: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
        run_build = parse_setuppy_commands()
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\bit_generator.pyx
      Processing numpy/random\mtrand.pyx
      Processing numpy/random\_bounded_integers.pyx.in
      Processing numpy/random\_common.pyx
      Processing numpy/random\_generator.pyx
      Processing numpy/random\_mt19937.pyx
      Processing numpy/random\_pcg64.pyx
      Processing numpy/random\_philox.pyx
      Processing numpy/random\_sfc64.pyx
      blas_opt_info:
      blas_mkl_info:
   

In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

<b style="color:darkorange">Question 1.</b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For your reference, the price of the option is $5.152$.

In [25]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $D_{0,t}V_t$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Question 1.1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [26]:
Bs = np.array([ 87.52900166,  87.80954309,  88.09098369,  88.37332635, 88.72752759,  89.1544435 ,  89.61925406,  90.19465661, 90.88274744,  91.75942312,  92.97867688,  94.85697075, 100.])

def exer_or_cont(i, S):
    return S <= Bs[i]

In [27]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 1000000)

# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in range(npaths):
        if exer_or_cont(i, paths[i][j]):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martinglae increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1501
Dual Price   (Upper Bound) = 5.1639


Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

<b style="color:darkorange">Question 2.</b> Consider pricing the Bermudan put option as in Question 1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then use the Broadie-Andersen algorithm to compute the corresponding upper bound.

For your convenience, an implementation of Longstaff-Schwartz algorithm is included in the cell below. It computes the regression coefficients at each exercise date.

In [28]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 10000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
betas_LS                                           # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.61200545,  1.32865118],
       [-0.52811734,  1.31018461],
       [-0.2993146 ,  1.25387479],
       [-0.20469928,  1.22419126],
       [-0.08698532,  1.17660602],
       [-0.06788037,  1.14835837],
       [-0.04283457,  1.12258675],
       [ 0.02663141,  1.07875206],
       [ 0.02979719,  1.07142193],
       [-0.00297595,  1.02962992],
       [-0.03726507,  0.99348487],
       [        nan,         nan]])

<b style="color:darkorange">Question 3.</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

For your convenience, an implementation of TVR algorithm is included in the cell below. It computes the regression coefficients at each exercise date. 

In [29]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 100000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)                      # compute values
betas_TVR                                                 # No regression needed at first and last time steps, return NaN

array([[        nan,         nan],
       [-0.44831651,  1.3194258 ],
       [-0.38839588,  1.29892731],
       [-0.29466453,  1.2693566 ],
       [-0.21138627,  1.23308337],
       [-0.11852737,  1.19579356],
       [-0.06142157,  1.16318152],
       [-0.02247265,  1.13104544],
       [ 0.00482802,  1.09930367],
       [ 0.01878937,  1.06453631],
       [ 0.01088555,  1.03560762],
       [-0.00870258,  1.00114663],
       [        nan,         nan]])

The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Question 4</b> (<b>Bermudan-Asian Call Option</b>). In Question 2.4 of Homework I, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

<b>(a).</b> Reuse the exercise strategy derived from the basis functions used in Question 2.4 of Homework I by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound. For your convenience, an implementation of Longstaff-Schwartz is included in the cell below. Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.

<b>(b).</b> As an alternative set of basis functions at time $t_n$, use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

<b>(c).</b> Compare the numerical duality gaps obtained from (a) and (b). Which set of basis functions does a better job at estimating the optimal exercise strategy?

In [30]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 100000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
vol_ = 0.1
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=float)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC # no need to estimate continuation values at the first date (not an exercise date) or the last date (always exercise)

array([[        nan,         nan],
       [ 0.25878656,  1.12597555],
       [-0.00372518,  1.10182677],
       [-0.19309499,  1.08738919],
       [-0.31767054,  1.07549266],
       [-0.37104361,  1.06223243],
       [-0.39173008,  1.05190202],
       [-0.38123969,  1.03960112],
       [-0.36580454,  1.03242928],
       [-0.32792397,  1.02552431],
       [-0.25803249,  1.01730498],
       [-0.17127565,  1.0077737 ],
       [        nan,         nan]])